In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pylab as plt

from statsmodels.tsa.stattools import adfuller
from numpy import log

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

## Data Import and Preview

In [2]:
raw_cal = pd.read_csv('calendar.csv')

In [3]:
raw_sales = pd.read_csv('sales_train_evaluation.csv')

In [4]:
raw_sales.shape

(30490, 1947)

In [5]:
cal = raw_cal
sales_cat = raw_sales.groupby(['cat_id']).sum().T
sales_cat["date"] = pd.date_range(start="2011-01-29", periods=len(sales_cat), freq='D')

In [6]:
result = adfuller(sales_cat["FOODS"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

ADF Statistic: -1.558381
p-value: 0.504459


In [8]:
# Original Series
fig, axes = plt.subplots(2, 2, sharex=True)
axes[0, 0].plot(sales_cat["FOODS"]); axes[0, 0].set_title('Original Series')
plot_acf(sales_cat["FOODS"], ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(sales_cat["FOODS"].diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(sales_cat["FOODS"].diff().dropna(), ax=axes[1, 1])

plt.show()

KeyboardInterrupt: 

In [ ]:
plt.plot(sales_cat["FOODS"])
plot_acf(sales_cat["FOODS"])